In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from sqlalchemy import create_engine
import pandas as pd
# import getpass


In [7]:
# password = getpass.getpass()
# !RSYNC_PASSWORD={password} rsync -avPR -e 'ssh -p 23263' mathias@gate08.aime.info:/home/mathias/workspace/pia-ic-web/./results.db ~/Downloads
# del password

mathias@gate08.aime.info's password: 


In [10]:
with create_engine('sqlite:////home/mathias/Downloads/results.db').connect() as conn: 
    result = pd.read_sql_table('result', conn)
    sample_collection = pd.read_sql_table('sample_collection', conn)


In [19]:
result.groupby('exp_type').apply(lambda result: (result.correct_exp == result.selected_exp).mean())

exp_type
0               1.0
1               1.0
2               1.0
drawn_curves    1.0
dtype: float64